In [1]:
import numpy as np
import deeplift
import evautils
from evautils import sequtils
from evautils import kerasutils
from evautils import dirutils
from evautils import impscoringutils
from __future__ import print_function
from collections import OrderedDict
import os
import h5py

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
REGION_SIZE = 400
CELL_LINE = 'H1ESC'
POS_PREFIX = CELL_LINE +'_' + str(REGION_SIZE)
MASTER_DIR='/users/eprakash/benchmarking/H1ESC/400bp_hg38'
DL_BASE_DIR=MASTER_DIR+'/deeplift'
PREPROCESSING_BASE_DIR = MASTER_DIR+'/preprocessing'
TRAINING_BASE_DIR=MASTER_DIR+'/training'
MOMMA_DRAGONN=TRAINING_BASE_DIR+'/'+'momma_dragonn'
IMPLANTED_POS_BED_FILE = PREPROCESSING_BASE_DIR + '/' + 'implanted_' + POS_PREFIX + '.bed.gz'
MODEL_PREFIX='record_1_model_XHjBt_'
MODEL=MOMMA_DRAGONN+'/examples/fasta_sequential_model/model_files/'+MODEL_PREFIX+'modelJson.json'
WEIGHTS=MOMMA_DRAGONN+'/examples/fasta_sequential_model/model_files/'+MODEL_PREFIX+'modelWeights.h5'

In [4]:
dirutils.createDir(DL_BASE_DIR)

In [5]:
data_filename_positive = IMPLANTED_POS_BED_FILE
labeled_sequences = sequtils.load_sequences_from_bedfile(data_filename_positive)
print("Got %d positive sequences" % len(labeled_sequences))
positive_labels = labeled_sequences.keys()
labels = labeled_sequences.keys()
sequences =labeled_sequences.values()
print("Sequences length: ", len(sequences))

#Loading /users/eprakash/benchmarking/H1ESC/400bp_hg38/preprocessing/implanted_H1ESC_400.bed.gz ...
#Loaded 96663 sequences from /users/eprakash/benchmarking/H1ESC/400bp_hg38/preprocessing/implanted_H1ESC_400.bed.gz
Got 96663 positive sequences
Sequences length:  96663


In [6]:
sequtils.removeUnsupportedChars(sequences, labels, labeled_sequences)

96663
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGAATTCTATCTGATGGCGTTGTCATTTTTGTGCGGTTTTTGTTTCCCGGTTGTGTTTTCGAGCGGCTGTATTGGATGGAATTGAGGGGTCTGCTAAGGTTGCTGCTTTTCTCTAGAGATTCTGGCTTTTAATGGGTGCAGCCGCTGTTTTTTTGTTGCTTCAGAGATGCGCATCGATGCCGCATTGGGGGCTAGAGCTTGCTCGGATTGTTTCCTGTAGCTGCCTATTCCGGATGCGTTTGCTTGTGGCGATCTCGTTTTCCAATCAGGCAGGGGAAAGGTTTTCCAGAGGAAAAGGTTGTTGCCAGTGGATTTGGTTCCGTTCGGTGCGTTGTTCTGGCTAGGTGGGGGGAGGGGCTGTCCCGTG
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGAATTCCTGCTGAAGGCACGCGCGTTGCCAGGCAATAGCTTCTTGGAGGGGCGCCCCTCTACTGCGCATGTGCGGGTTTCCAGGCAGGGATCGGTCGTTCTCCTCTAGCCCAGGCGGAGCTCTGTCACGTTTTTCCCCGCGTTCGGTCTTCGGGTTGGCGGTCCCAACACACCGCGAAACAAGCCGGCCCAGGATTTCTCCCATGTCAGGGCCTTTCTGCGGCTAGTGAGCCCACTCTCTTCGCCCGTGGTCCAGTAGGCTTCCCACCCTTGAGAGGGCTCTT
TATGTAGTAACTCCTATATACCATTAATATATATATAATATGTCCATTGTATCATAACCTGTATGTAAATCTATTAACAACCATCTATAAATGAGTAATGTACCATGTAAGTCAGTAGTGTGTAATAAAAGACTAAACTAATATACAGTACAACAACAAATATACAGAGTAAATAAAAATTCAATAACCCAA

In [7]:
onehot_data = np.array([sequtils.one_hot_encode_along_channel_axis(seq) for seq in sequences])
print(onehot_data.shape)

(96660, 400, 4)


In [8]:
keras_model=kerasutils.load_keras_model_using_json(MODEL, WEIGHTS)
keras_model.summary()

Loading Keras JSON model from file /users/eprakash/benchmarking/H1ESC/400bp_hg38/training/momma_dragonn/examples/fasta_sequential_model/model_files/record_1_model_XHjBt_modelJson.json
Loading Keras model weights from file /users/eprakash/benchmarking/H1ESC/400bp_hg38/training/momma_dragonn/examples/fasta_sequential_model/model_files/record_1_model_XHjBt_modelWeights.h5
Successfully loaded
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 393, 320)          10560     
_________________________________________________________________
activation_1 (Activation)    (None, 393, 320)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 386, 320)          819520    
_________________________________________________________________
activation_2 (Activation)    (None, 386, 320)          0  

In [9]:
preds = keras_model.predict(onehot_data)
preds.shape

(96660, 1)

In [10]:
top_1k_pos_labels, top_pos_seqs=kerasutils.getTopNumPos(labels, labeled_sequences, preds, 1000)

96660
96660


In [11]:
h5f = h5py.File(DL_BASE_DIR+'/'+POS_PREFIX+'_top_1k_pos_labels.h5', 'w')
h5f.create_dataset('labels', data=top_1k_pos_labels)
h5f.close()

In [12]:
method_to_model=kerasutils.prepareDLModel(WEIGHTS, MODEL)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the preceding linear layer is 2 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
For layer 7 the preceding linear layer is 6 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 9 the preceding linear layer is 8 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
For layer 13 the preceding lin

In [13]:
#make sure predictions are the same as the original model
from deeplift.util import compile_func
model_to_test = method_to_model['rescale_conv_revealcancel_fc']
kerasutils.sanityCheck(model_to_test, onehot_data, keras_model)

('maximum difference in predictions:', 0.0)


In [14]:
method_to_scoring_func = impscoringutils.compileScoringFunctions(method_to_model)
print("Done!")

Compiling scoring functions
Compiling scoring function for: rescale_conv_revealcancel_fc
Compiling scoring function for: rescale_all_layers
Compiling scoring function for: revealcancel_all_layers
Compiling scoring function for: grad_times_inp
Compiling scoring function for: guided_backprop
Compiling integrated gradients scoring functions
Done!


In [15]:
method_to_task_to_scores = OrderedDict()
all_zeroes_methods=['grad_times_inp', 'rescale_all_layers']
avg_gc_methods=['rescale_all_layers']
multiref_methods=['rescale_all_layers', 'rescale_conv_revealcancel_fc']
ig=['integrated_gradients10']
h5f = h5py.File(DL_BASE_DIR+'/'+POS_PREFIX+'_dl_scores.h5', 'w')
h5f.create_dataset("labels", data=labels)

<HDF5 dataset "labels": shape (96660,), type "|S57">

In [16]:
impscoringutils.flatRefScore(method_to_task_to_scores, method_to_scoring_func, ig, onehot_data, 0)
print("Done!")
for meth in method_to_task_to_scores.keys():
    print("Storing scores for " + str(meth))
    h5f.create_dataset(meth, data=method_to_task_to_scores[meth][0])
method_to_task_to_scores.clear()

['rescale_conv_revealcancel_fc', 'rescale_all_layers', 'revealcancel_all_layers', 'grad_times_inp', 'guided_backprop', 'integrated_gradients10']
('on method', 'rescale_conv_revealcancel_fc')
('scorefunc ', <function func at 0x7f3b94416488>)
('on method', 'rescale_all_layers')
('scorefunc ', <function func at 0x7f3b94416e60>)
('on method', 'revealcancel_all_layers')
('scorefunc ', <function func at 0x7f3b8d73af50>)
('on method', 'grad_times_inp')
('scorefunc ', <function func at 0x7f3b8ce04140>)
('on method', 'guided_backprop')
('scorefunc ', <function func at 0x7f3b8c69a938>)
('on method', 'integrated_gradients10')
('scorefunc ', <function compute_integrated_gradients at 0x7f3b7bf8e050>)
On method variation: integrated_gradients10_all_zeros_ref
Done!
Storing scores for integrated_gradients10_all_zeros_ref


In [ ]:
impscoringutils.flatRefScore(method_to_task_to_scores, method_to_scoring_func, all_zeroes_methods, onehot_data, 0)
print("Done!")
for meth in method_to_task_to_scores.keys():
    print("Storing scores for " + str(meth))
    h5f.create_dataset(meth, data=method_to_task_to_scores[meth][0])
method_to_task_to_scores.clear()

['rescale_conv_revealcancel_fc', 'rescale_all_layers', 'revealcancel_all_layers', 'grad_times_inp', 'guided_backprop', 'integrated_gradients10']
('on method', 'rescale_conv_revealcancel_fc')
('scorefunc ', <function func at 0x7f3b94416488>)
('on method', 'rescale_all_layers')
('scorefunc ', <function func at 0x7f3b94416e60>)
On method variation: rescale_all_layers_all_zeros_ref
('on method', 'revealcancel_all_layers')
('scorefunc ', <function func at 0x7f3b8d73af50>)
('on method', 'grad_times_inp')
('scorefunc ', <function func at 0x7f3b8ce04140>)
On method variation: grad_times_inp_all_zeros_ref
('on method', 'guided_backprop')
('scorefunc ', <function func at 0x7f3b8c69a938>)
('on method', 'integrated_gradients10')
('scorefunc ', <function compute_integrated_gradients at 0x7f3b7bf8e050>)
Done!
Storing scores for rescale_all_layers_all_zeros_ref
Storing scores for grad_times_inp_all_zeros_ref


In [ ]:
impscoringutils.flatRefScore(method_to_task_to_scores, method_to_scoring_func, ig, onehot_data, 1)
print("Done!")
for meth in method_to_task_to_scores.keys():
    print("Storing scores for " + str(meth))
    h5f.create_dataset(meth, data=method_to_task_to_scores[meth][0])
method_to_task_to_scores.clear()

[0.2195576  0.28032617 0.28106109 0.21905514]
OrderedDict([('A', 0.21955759879991724), ('C', 0.2803261690461411), ('G', 0.28106109042002897), ('T', 0.2190551417339127)])
['rescale_conv_revealcancel_fc', 'rescale_all_layers', 'revealcancel_all_layers', 'grad_times_inp', 'guided_backprop', 'integrated_gradients10']
('on method', 'rescale_conv_revealcancel_fc')
('scorefunc ', <function func at 0x7f3b94416488>)
('on method', 'rescale_all_layers')
('scorefunc ', <function func at 0x7f3b94416e60>)
('on method', 'revealcancel_all_layers')
('scorefunc ', <function func at 0x7f3b8d73af50>)
('on method', 'grad_times_inp')
('scorefunc ', <function func at 0x7f3b8ce04140>)
('on method', 'guided_backprop')
('scorefunc ', <function func at 0x7f3b8c69a938>)
('on method', 'integrated_gradients10')
('scorefunc ', <function compute_integrated_gradients at 0x7f3b7bf8e050>)
On method variation: integrated_gradients10_avg_gc_ref


In [ ]:
impscoringutils.flatRefScore(method_to_task_to_scores, method_to_scoring_func, avg_gc_methods, onehot_data, 1)
print("Done!")
for meth in method_to_task_to_scores.keys():
    print("Storing scores for " + str(meth))
    h5f.create_dataset(meth, data=method_to_task_to_scores[meth][0])
method_to_task_to_scores.clear()

In [ ]:
impscoringutils.multirefScore(method_to_task_to_scores, method_to_scoring_func, ig, top_pos_seqs)
print("Done!")
for meth in method_to_task_to_scores.keys():
    print("Storing scores for " + str(meth))
    h5f.create_dataset(meth, data=method_to_task_to_scores[meth][0])
method_to_task_to_scores.clear()

In [ ]:
impscoringutils.multirefScore(method_to_task_to_scores, method_to_scoring_func, multiref_methods, sequences)
print("Done!")

In [ ]:
for meth in method_to_task_to_scores.keys():
    print("Storing scores for " + str(meth))
    h5f.create_dataset(meth, data=method_to_task_to_scores[meth][0])
method_to_task_to_scores.clear()
h5f.close()